In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling layer #2
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



def main(unused_argv):

  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
  mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)
  

if __name__ == "__main__":
  tf.app.run()


Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.09541416  0.11239035  0.08497782  0.10926841  0.10117733  0.08583753
   0.09634782  0.10485031  0.09778686  0.11194938]
 [ 0.09890205  0.11326732  0.08249144  0.10361141  0.10620973  0.09251516
   0.10940047  0.09219741  0.09630229  0.1051027 ]
 [ 0.09825759  0.10674597  0.09763835  0.10967374

INFO:tensorflow:loss = 2.30948, step = 1
INFO:tensorflow:probabilities = [[ 0.08896658  0.10620184  0.07438093  0.12286127  0.10316802  0.09040038
   0.09767068  0.09335484  0.09878856  0.12420695]
 [ 0.08114499  0.11203656  0.08041549  0.09986547  0.1090045   0.09993751
   0.10147359  0.09418523  0.10226509  0.11967161]
 [ 0.0915135   0.11916785  0.09106634  0.10540076  0.11106803  0.08984501
   0.08967824  0.07550567  0.09635004  0.13040462]
 [ 0.09325644  0.11504805  0.08985751  0.10542063  0.10215656  0.09352102
   0.10601344  0.09220363  0.09751213  0.10501057]
 [ 0.09495007  0.10486781  0.09061056  0.1046052   0.10736576  0.09737927
   0.10982328  0.08350004  0.09623049  0.11066741]
 [ 0.09162541  0.12574719  0.08841335  0.09421203  0.11170569  0.08520549
   0.09787992  0.10714883  0.09467142  0.10339069]
 [ 0.09645891  0.09487525  0.09857552  0.10696562  0.09877013  0.0953512
   0.1088422   0.10056762  0.09414492  0.10544855]
 [ 0.09615505  0.09380266  0.09351125  0.10742914  0.

INFO:tensorflow:global_step/sec: 2.89532
INFO:tensorflow:probabilities = [[ 0.10698508  0.11339204  0.09380939  0.10438129  0.09414726  0.09606508
   0.10502059  0.09341791  0.08906998  0.10371136]
 [ 0.09963374  0.10203075  0.08064776  0.10842565  0.11251257  0.07652359
   0.10658652  0.1106181   0.08743688  0.11558443]
 [ 0.10326181  0.09215282  0.09325918  0.10681674  0.08932501  0.09936564
   0.10002413  0.10431813  0.10353788  0.10793869]
 [ 0.08678953  0.1187337   0.09337487  0.10360181  0.10337269  0.08381018
   0.10828807  0.09418432  0.09763459  0.11021025]
 [ 0.10344981  0.10950033  0.08549362  0.10882918  0.09980935  0.09486405
   0.10104504  0.10473951  0.08149467  0.11077452]
 [ 0.095006    0.11029939  0.08939689  0.11334519  0.10462524  0.0889084
   0.10776422  0.09685077  0.08928207  0.10452179]
 [ 0.08957711  0.10805536  0.08932629  0.10303304  0.10378668  0.1074247
   0.10478565  0.08899979  0.10445782  0.10055353]
 [ 0.10185584  0.10914557  0.0831538   0.11603308  0.0

INFO:tensorflow:loss = 2.28038, step = 101 (34.563 sec)
INFO:tensorflow:probabilities = [[ 0.089987    0.10283964  0.08712915  0.09254079  0.10101061  0.09045163
   0.12742187  0.08910114  0.11034569  0.10917243]
 [ 0.08527485  0.09796111  0.09116887  0.11465506  0.09871823  0.08643647
   0.11357238  0.10328434  0.08922777  0.11970073]
 [ 0.09542376  0.1002669   0.11165116  0.08752329  0.11523065  0.10518003
   0.09803606  0.08009755  0.10524457  0.10134596]
 [ 0.10039753  0.10249481  0.09352847  0.11289316  0.09368806  0.08772523
   0.10113645  0.09339119  0.09730612  0.1174389 ]
 [ 0.08428068  0.10637803  0.08295316  0.1113589   0.10000976  0.09306239
   0.11648587  0.08788277  0.10594831  0.11164014]
 [ 0.09276494  0.10413447  0.09732079  0.11162274  0.09978455  0.08201073
   0.09210678  0.09595677  0.10929149  0.11500677]
 [ 0.08785176  0.10062651  0.09233879  0.10448054  0.10324211  0.09408378
   0.1134648   0.09351636  0.11057611  0.09981918]
 [ 0.09342428  0.10893171  0.08572275

INFO:tensorflow:global_step/sec: 2.65356
INFO:tensorflow:probabilities = [[ 0.09391235  0.11186327  0.0890611   0.09885612  0.09878874  0.09238901
   0.1011866   0.09715125  0.10786846  0.10892307]
 [ 0.09536514  0.11421434  0.08557478  0.10249125  0.10461271  0.0935758
   0.10049132  0.1002105   0.10147998  0.10198427]
 [ 0.10418368  0.09973998  0.09117063  0.10576788  0.09846945  0.09986702
   0.1073105   0.09159517  0.09302527  0.1088704 ]
 [ 0.11646423  0.09040719  0.10121354  0.10071495  0.09376664  0.09982801
   0.09865664  0.09287045  0.09996237  0.1061159 ]
 [ 0.10295324  0.10275903  0.08886646  0.11848209  0.09416007  0.1083985
   0.10557023  0.08141363  0.09841625  0.09898046]
 [ 0.08958763  0.10568118  0.08774417  0.11374182  0.09987398  0.07754401
   0.10575496  0.09493601  0.09818511  0.12695116]
 [ 0.09987673  0.10744631  0.09700287  0.10762788  0.0916381   0.09597507
   0.09445226  0.10176466  0.09626044  0.10795572]
 [ 0.10061043  0.10697515  0.09149323  0.10132842  0.0

...

INFO:tensorflow:loss = 0.431083, step = 4201 (52.910 sec)
INFO:tensorflow:probabilities = [[ 0.00038718  0.88744026  0.04362808  0.00805434  0.00102516  0.00250607
   0.03991521  0.00002764  0.01651498  0.00050123]
 [ 0.00292437  0.00000016  0.00849088  0.00003253  0.00004124  0.00062655
   0.98704773  0.00000029  0.00079656  0.00003954]
 [ 0.00005083  0.00005746  0.12310769  0.84545183  0.00015107  0.00229278
   0.0000545   0.00024308  0.02719863  0.00139208]
 [ 0.00050798  0.00161775  0.98560613  0.00053604  0.00022625  0.00000492
   0.01105391  0.00012155  0.00026207  0.00006336]
 [ 0.04355261  0.00027297  0.88022995  0.00087773  0.0008859   0.00566645
   0.06404781  0.00008659  0.004108    0.00027199]
 [ 0.0002788   0.00044161  0.00406515  0.00102752  0.19148621  0.00557503
   0.02716366  0.00053529  0.74444199  0.02498471]
 [ 0.00004388  0.0001359   0.99521965  0.00059487  0.0000049   0.00004089
   0.00084449  0.00002007  0.00306298  0.00003244]
 [ 0.01033426  0.03381961  0.021031

INFO:tensorflow:global_step/sec: 1.90197
INFO:tensorflow:probabilities = [[ 0.00313977  0.00000026  0.00561582  0.0000269   0.00008046  0.0008685
   0.98959643  0.00000247  0.00052592  0.00014343]
 [ 0.00005778  0.00183825  0.03398691  0.04775951  0.0006881   0.00368714
   0.00108662  0.00133096  0.90015513  0.0094096 ]
 [ 0.00127608  0.00005419  0.00209593  0.00070249  0.03664116  0.00089782
   0.95572513  0.00012067  0.00106353  0.00142298]
 [ 0.0077283   0.00362488  0.00216608  0.63893509  0.00113476  0.01829735
   0.00019529  0.12842135  0.13781102  0.06168587]
 [ 0.01833996  0.03667473  0.02084335  0.0071804   0.00701693  0.02421537
   0.00846994  0.0026511   0.84939009  0.02521816]
 [ 0.01015809  0.02364101  0.36974856  0.06606852  0.00331994  0.00414777
   0.40528142  0.00251345  0.11117404  0.00394718]
 [ 0.00010562  0.98617768  0.00316457  0.00250293  0.00019451  0.000319
   0.00087385  0.00120091  0.00482993  0.00063117]
 [ 0.00067283  0.0001524   0.0000969   0.00642167  0.00

INFO:tensorflow:loss = 0.337616, step = 4301 (52.577 sec)
INFO:tensorflow:probabilities = [[ 0.0024901   0.00001082  0.78300411  0.19720213  0.00000011  0.00130886
   0.00013799  0.00966979  0.00591104  0.00026522]
 [ 0.00019206  0.0001841   0.00012781  0.00870866  0.10496827  0.0007329
   0.00007226  0.22667292  0.01021252  0.64812857]
 [ 0.0289454   0.00003471  0.00052303  0.00016252  0.00031207  0.94153196
   0.00129709  0.00374695  0.02256397  0.00088245]
 [ 0.02613299  0.00068138  0.00367331  0.00394615  0.00351219  0.19291353
   0.75398117  0.00002003  0.01336698  0.00177228]
 [ 0.00002715  0.00049638  0.00026587  0.00288857  0.1907775   0.00208118
   0.00019167  0.07312124  0.04507312  0.68507725]
 [ 0.0000411   0.54655755  0.00180352  0.05736067  0.00113781  0.00258443
   0.00073601  0.34299654  0.00523732  0.04154506]
 [ 0.00059946  0.0003981   0.00203087  0.97692114  0.00001916  0.01744459
   0.00006308  0.00008684  0.00207343  0.00036321]
 [ 0.99543142  0.          0.0000261

INFO:tensorflow:Saving checkpoints for 4373 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:global_step/sec: 0.00922482
INFO:tensorflow:probabilities = [[ 0.00264125  0.06866451  0.17357706  0.00536217  0.00027701  0.07052001
   0.67565906  0.00002785  0.00321562  0.00005545]
 [ 0.01990555  0.00001645  0.95114326  0.00188558  0.00050202  0.00075934
   0.00520856  0.00011066  0.02000038  0.00046817]
 [ 0.00009845  0.99314535  0.00203751  0.00118107  0.000096    0.00014706
   0.00057336  0.00047718  0.00162851  0.00061555]
 [ 0.95893663  0.00004979  0.00055783  0.00543395  0.00004335  0.00580876
   0.0206131   0.00002863  0.00770225  0.00082573]
 [ 0.00115008  0.00029166  0.05880261  0.00007244  0.02750169  0.00104059
   0.90269572  0.00000019  0.00818893  0.00025608]
 [ 0.87868196  0.00012621  0.0034601   0.01281095  0.00300414  0.08798437
   0.00319127  0.00061554  0.00612085  0.00400461]
 [ 0.00059216  0.94571698  0.01792163  0.00386664  0.00113599  0.0005395
   0.00146965  

INFO:tensorflow:loss = 0.339945, step = 4401 (10840.316 sec)
INFO:tensorflow:Saving checkpoints for 4420 into /tmp/mnist_convnet_model\model.ckpt.
